In [203]:
import pandas as pd
import numpy as np
from math import radians, cos, sin, asin, sqrt
pd.set_option('display.max_columns', None)
import plotly.express as px
import plotly.graph_objects as go


In [306]:
def haversine(x):
    """
    Calculate the great circle distance between two points 
    on the earth (specified in decimal degrees)
    """
    
    lat1 = float(x.split(",")[0])
    lon1 = float(x.split(",")[1])
    
    global api_20km
    api_20km = []
    
    for item in coord_list:
        lat2 = item["lat"]
        lon2 = item["lon"]
    
        # convert decimal degrees to radians 
        rlon1, rlat1, rlon2, rlat2 = map(radians, [lon1, lat1, lon2, lat2])

        # haversine formula 
        dlon = rlon2 - rlon1 
        dlat = rlat2 - rlat1 
        a = sin(dlat/2)**2 + cos(rlat1) * cos(rlat2) * sin(dlon/2)**2
        c = 2 * asin(sqrt(a)) 
        r = 6371 # Radius of earth in kilometers. Use 3956 for miles
        dist = c * r
        
        if dist <= 20:
            
            api_20km.append(item["api"])
            
    return api_20km



In [312]:
# Import monthly injection volumes - filtered wells that are inside the Area of Interest
df_filtered_monthly = pd.read_csv("../oklahoma_earthquakes_largefiles/for_analysis/df_filtered_monthly.csv", low_memory=False)

In [314]:
del df_filtered_monthly['APIyear.1']

In [315]:
df_filtered_monthly

,API,API_year,APIyear,year,Operator_Name,Operator_Number,Well_Name,Well_Number,Latitude,Longitude,County,Total_Depth,Formation_Name,Jan_Vol,Feb_Vol,Mar_Vol,Apr_Vol,May_Vol,Jun_Vol,Jul_Vol,Aug_Vol,Sep_Vol,Oct_Vol,Nov_Vol,Dec_Vol,year_volume,data_origin
0,3504724528,3504724528-2011,35047245282011,2011,SANDRIDGE EXPLORATION & PRODUCTION LLC,NaN,LOMA SWD,1-1,36.504361,-97.890361,GARFIELD,8500.0,ARBUCKLE,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3189.00000,71226.00000,38259.00000,112674.00000,monthly
1,3504724516,3504724516-2011,35047245162011,2011,SK PLYMOUTH LLC,NaN,TOEWS,1-15 SWD,36.562985,-97.617946,GARFIELD,7503.0,ARBUCKLE,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,39676.89063,35082.08984,56822.87109,131581.85156,monthly
2,3504724485,3504724485-2011,35047244852011,2011,KIRKPATRICK OIL COMPANY INC,NaN,MARSHALL EAST SWD,1,36.165073,-97.602745,GARFIELD,7439.0,ARBUCKLE|WILCOX,19769.0,20939.0,42513.0,28752.0,30183.0,29888.0,30605.0,27052.0,25727.0,27502.00000,25197.00000,25491.00000,333618.00000,monthly
3,3504724444,3504724444-2011,35047244442011,2011,ARROWHEAD ENERGY INC,NaN,VENCL SWD,2-26,36.532275,-97.696179,GARFIELD,7212.0,NaN,7440.0,6720.0,7440.0,7200.0,7440.0,7200.0,7440.0,7440.0,7200.0,7440.00000,7200.00000,7440.00000,87600.00000,monthly
4,3504724404,3504724404-2011,35047244042011,2011,M M ENERGY INC,NaN,GREGG,1,36.355337,-97.562891,GARFIELD,6000.0,NaN,24440.0,21785.0,24840.0,23400.0,22175.0,12950.0,25790.0,24710.0,26370.0,28190.00000,27800.00000,29140.00000,291590.00000,monthly
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7009,3503729201,3503729201-2020,35037292012020,2020,BAUGH ALLAN L,NaN,FOUNTAIN SWD,1,35.902091,-96.584933,CREEK,4300.0,ARBUCKLE,0.0,0.0,0.0,0.0,0.0,0.0,4020.0,2990.0,1900.0,2270.00000,3140.00000,3445.00000,17765.00000,monthly
7010,3503729213,3503729213-2020,35037292132020,2020,CIMARRON RIVER OPERATING CO,NaN,E. YAHOLA,2A,36.057570,-96.596340,CREEK,3450.0,ARBUCKLE,4450.0,40369.0,41212.0,40127.0,41168.0,36634.0,39317.0,39384.0,36742.0,38065.00000,39133.00000,39818.00000,436419.00000,monthly
7011,3503729215,3503729215-2020,35037292152020,2020,MID-CON ENERGY OPERATING LLC,NaN,MCBRIDE,35,36.050040,-96.599400,CREEK,3400.0,ARBUCKLE,215120.0,181020.0,95300.0,45220.0,90330.0,47310.0,40980.0,50920.0,37460.0,43550.00000,42340.00000,40060.00000,929610.00000,monthly
7012,3503736202,3503736202-2020,35037362022020,2020,PETCO PETROLEUM CORP,NaN,BEN WACOCHE,4,36.027504,-96.598293,CREEK,2680.0,ARBUCKLE,91791.0,78809.0,90007.0,90098.0,78189.0,75234.0,75509.0,0.0,0.0,0.00000,0.00000,0.00000,579637.00000,monthly


In [316]:
# get relevant columnes, add to list to iterate through
api_list = df_filtered_monthly.API.to_list()
# api_year_list = monthly_inj_df.API_year.to_list()
lat_list = df_filtered_monthly.Latitude.to_list()
lon_list = df_filtered_monthly.Longitude.to_list()

coord_list = []
api_tracker = []

for count, value in enumerate(api_list):
    if value not in api_tracker:
        api_tracker.append(value)
        coord_list.append({"api": value, "lat": lat_list[count], "lon": lon_list[count]})

In [317]:
# opens earthquake data
eq_df = pd.read_csv('../oklahoma_earthquakes_largefiles/usgs_cleaned/usgs_eqs_reduced_data.csv', low_memory=False)
# converting date fields to date-time dtype
eq_df.iloc[:, 10] = eq_df.iloc[:, 10].apply(pd.to_datetime)

In [318]:
eq_df

,id,time,mag,magType,cdi,place,status,latitude,longitude,depth,date
0,us2000iz6u,1546166814510,2.2,mb_lg,NaN,"6 km ESE of Cleo Springs, Oklahoma",reviewed,36.3818,-98.3804,6.01,2018-12-30 10:46:54.510
1,us2000iz51,1546159833730,2.0,mb_lg,NaN,"5 km ESE of Cleo Springs, Oklahoma",reviewed,36.3917,-98.3808,6.28,2018-12-30 08:50:33.730
2,us2000iz4w,1546158498840,2.2,ml,NaN,"13 km NNW of Pawnee, Oklahoma",reviewed,36.4443,-96.8762,5.36,2018-12-30 08:28:18.840
3,us2000iyvs,1546073324730,2.5,mb_lg,3.3,"5 km WSW of Arcadia, Oklahoma",reviewed,35.6547,-97.3898,5.00,2018-12-29 08:48:44.730
4,us2000iyvq,1546073222930,2.2,mb_lg,NaN,"6 km W of Arcadia, Oklahoma",reviewed,35.6747,-97.3990,7.42,2018-12-29 08:47:02.930
...,...,...,...,...,...,...,...,...,...,...,...
20619,usp000gucq,1235535255330,3.3,mblg,3.8,"4 km SE of Ashland, Oklahoma",reviewed,34.7350,-96.0360,5.00,2009-02-25 04:14:15.330
20620,usp000gu9h,1235295786750,3.2,mblg,3.8,"2 km S of Lahoma, Oklahoma",reviewed,36.3690,-98.0870,5.00,2009-02-22 09:43:06.750
20621,usp000gtdu,1233656590010,3.1,mblg,2.2,"2 km SSE of Centrahoma, Oklahoma",reviewed,34.5890,-96.3400,5.00,2009-02-03 10:23:10.010
20622,usp000gt8m,1233277072380,2.4,mblg,2.9,"4 km NW of Chandler, Oklahoma",reviewed,35.7240,-96.9200,5.00,2009-01-30 00:57:52.380


In [319]:
eq_df['lat/lon'] = eq_df.latitude.astype(str) + "," + eq_df.longitude.astype(str)
eq_df.head()

,id,time,mag,magType,cdi,place,status,latitude,longitude,depth,date,lat/lon
0,us2000iz6u,1546166814510,2.2,mb_lg,NaN,"6 km ESE of Cleo Springs, Oklahoma",reviewed,36.3818,-98.3804,6.01,2018-12-30 10:46:54.510,"36.3818,-98.3804"
1,us2000iz51,1546159833730,2.0,mb_lg,NaN,"5 km ESE of Cleo Springs, Oklahoma",reviewed,36.3917,-98.3808,6.28,2018-12-30 08:50:33.730,"36.3917,-98.3808"
2,us2000iz4w,1546158498840,2.2,ml,NaN,"13 km NNW of Pawnee, Oklahoma",reviewed,36.4443,-96.8762,5.36,2018-12-30 08:28:18.840,"36.4443,-96.8762"
3,us2000iyvs,1546073324730,2.5,mb_lg,3.3,"5 km WSW of Arcadia, Oklahoma",reviewed,35.6547,-97.3898,5.00,2018-12-29 08:48:44.730,"35.6547,-97.3898"
4,us2000iyvq,1546073222930,2.2,mb_lg,NaN,"6 km W of Arcadia, Oklahoma",reviewed,35.6747,-97.3990,7.42,2018-12-29 08:47:02.930,"35.6747,-97.399"


In [320]:
# Get list of well APIs within 20km radius
eq_df["nearby_wells"] = eq_df["lat/lon"].apply(haversine)

In [321]:
# Get number of wells nearby
eq_df["num_wells"] = eq_df.nearby_wells.apply(lambda x: len(x))

In [322]:
eq_df_20km = eq_df[eq_df.num_wells > 0]
# eq_df_20km = eq_df_20km[eq_df_20km.mag > 3]
eq_df_20km

,id,time,mag,magType,cdi,place,status,latitude,longitude,depth,date,lat/lon,nearby_wells,num_wells
0,us2000iz6u,1546166814510,2.2,mb_lg,NaN,"6 km ESE of Cleo Springs, Oklahoma",reviewed,36.3818,-98.3804,6.01,2018-12-30 10:46:54.510,"36.3818,-98.3804","[3509321030, 3500322102, 3500322302, 350032254...",5
1,us2000iz51,1546159833730,2.0,mb_lg,NaN,"5 km ESE of Cleo Springs, Oklahoma",reviewed,36.3917,-98.3808,6.28,2018-12-30 08:50:33.730,"36.3917,-98.3808","[3509321030, 3500322102, 3500322302, 350032254...",5
2,us2000iz4w,1546158498840,2.2,ml,NaN,"13 km NNW of Pawnee, Oklahoma",reviewed,36.4443,-96.8762,5.36,2018-12-30 08:28:18.840,"36.4443,-96.8762","[3511700179, 3511704379, 3511720132, 351170108...",23
3,us2000iyvs,1546073324730,2.5,mb_lg,3.3,"5 km WSW of Arcadia, Oklahoma",reviewed,35.6547,-97.3898,5.00,2018-12-29 08:48:44.730,"35.6547,-97.3898","[3510900109, 3510920163, 3510921577, 351092199...",11
4,us2000iyvq,1546073222930,2.2,mb_lg,NaN,"6 km W of Arcadia, Oklahoma",reviewed,35.6747,-97.3990,7.42,2018-12-29 08:47:02.930,"35.6747,-97.399","[3510900109, 3510920163, 3510921577, 351092199...",10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20616,usp000gv37,1237311697000,2.5,mblg,3.6,"0 km SSE of Smith Village, Oklahoma",reviewed,35.4460,-97.4560,5.00,2009-03-17 17:41:37.000,"35.446,-97.456","[3510920077, 3510920193, 3510922128, 351092217...",17
20617,usp000gv36,1237310692000,2.7,mblg,3.3,"0 km SSE of Smith Village, Oklahoma",reviewed,35.4460,-97.4560,5.00,2009-03-17 17:24:52.000,"35.446,-97.456","[3510920077, 3510920193, 3510922128, 351092217...",17
20618,usp000gurq,1236517599940,3.4,mblg,4.0,"0 km SSE of Smith Village, Oklahoma",reviewed,35.4460,-97.4560,5.00,2009-03-08 13:06:39.940,"35.446,-97.456","[3510920077, 3510920193, 3510922128, 351092217...",17
20620,usp000gu9h,1235295786750,3.2,mblg,3.8,"2 km S of Lahoma, Oklahoma",reviewed,36.3690,-98.0870,5.00,2009-02-22 09:43:06.750,"36.369,-98.087","[3504724403, 3509323734, 3509323729, 350472461...",6


In [328]:
eq_df_20km['year_month'] = eq_df_20km.date.astype(str).apply(lambda row: row[0:7])

eq_df_20km['year_month_day'] = eq_df_20km.date.astype(str).apply(lambda row: row[0:10])

/Users/curtissmith/opt/anaconda3/envs/PythonData/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/Users/curtissmith/opt/anaconda3/envs/PythonData/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [329]:
eq_df_20km

,id,time,mag,magType,cdi,place,status,latitude,longitude,depth,date,lat/lon,nearby_wells,num_wells,year_month,year_month_day
0,us2000iz6u,1546166814510,2.2,mb_lg,NaN,"6 km ESE of Cleo Springs, Oklahoma",reviewed,36.3818,-98.3804,6.01,2018-12-30 10:46:54.510,"36.3818,-98.3804","[3509321030, 3500322102, 3500322302, 350032254...",5,2018-12,2018-12-30
1,us2000iz51,1546159833730,2.0,mb_lg,NaN,"5 km ESE of Cleo Springs, Oklahoma",reviewed,36.3917,-98.3808,6.28,2018-12-30 08:50:33.730,"36.3917,-98.3808","[3509321030, 3500322102, 3500322302, 350032254...",5,2018-12,2018-12-30
2,us2000iz4w,1546158498840,2.2,ml,NaN,"13 km NNW of Pawnee, Oklahoma",reviewed,36.4443,-96.8762,5.36,2018-12-30 08:28:18.840,"36.4443,-96.8762","[3511700179, 3511704379, 3511720132, 351170108...",23,2018-12,2018-12-30
3,us2000iyvs,1546073324730,2.5,mb_lg,3.3,"5 km WSW of Arcadia, Oklahoma",reviewed,35.6547,-97.3898,5.00,2018-12-29 08:48:44.730,"35.6547,-97.3898","[3510900109, 3510920163, 3510921577, 351092199...",11,2018-12,2018-12-29
4,us2000iyvq,1546073222930,2.2,mb_lg,NaN,"6 km W of Arcadia, Oklahoma",reviewed,35.6747,-97.3990,7.42,2018-12-29 08:47:02.930,"35.6747,-97.399","[3510900109, 3510920163, 3510921577, 351092199...",10,2018-12,2018-12-29
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20616,usp000gv37,1237311697000,2.5,mblg,3.6,"0 km SSE of Smith Village, Oklahoma",reviewed,35.4460,-97.4560,5.00,2009-03-17 17:41:37.000,"35.446,-97.456","[3510920077, 3510920193, 3510922128, 351092217...",17,2009-03,2009-03-17
20617,usp000gv36,1237310692000,2.7,mblg,3.3,"0 km SSE of Smith Village, Oklahoma",reviewed,35.4460,-97.4560,5.00,2009-03-17 17:24:52.000,"35.446,-97.456","[3510920077, 3510920193, 3510922128, 351092217...",17,2009-03,2009-03-17
20618,usp000gurq,1236517599940,3.4,mblg,4.0,"0 km SSE of Smith Village, Oklahoma",reviewed,35.4460,-97.4560,5.00,2009-03-08 13:06:39.940,"35.446,-97.456","[3510920077, 3510920193, 3510922128, 351092217...",17,2009-03,2009-03-08
20620,usp000gu9h,1235295786750,3.2,mblg,3.8,"2 km S of Lahoma, Oklahoma",reviewed,36.3690,-98.0870,5.00,2009-02-22 09:43:06.750,"36.369,-98.087","[3504724403, 3509323734, 3509323729, 350472461...",6,2009-02,2009-02-22


In [330]:
temp = eq_df_20km.loc[eq_df_20km.year_month == '2016-02']
temp.sort_values("date").head(50)


,id,time,mag,magType,cdi,place,status,latitude,longitude,depth,date,lat/lon,nearby_wells,num_wells,year_month,year_month_day
4135,us20004w8b,1454292325700,2.5,ml,NaN,"4 km SW of Luther, Oklahoma",reviewed,35.6382,-97.2336,5.967,2016-02-01 02:05:25.700,"35.6382,-97.2336","[3508102053, 3508123907, 3508124016, 350812384...",21,2016-02,2016-02-01
4134,us20004w9e,1454319923600,3.0,ml,3.8,"1 km NNW of Stillwater, Oklahoma",reviewed,36.1242,-97.0618,6.929,2016-02-01 09:45:23.600,"36.1242,-97.0618","[3511921412, 3511922920, 3511922725, 351192038...",55,2016-02,2016-02-01
4133,us20004wb3,1454349916700,2.7,ml,NaN,"13 km SSW of Wakita, Oklahoma",reviewed,36.7825,-98.0048,6.246,2016-02-01 18:05:16.700,"36.7825,-98.0048","[3505322795, 3505322791, 3505322796, 350532248...",40,2016-02,2016-02-01
4132,us20004wcb,1454351444500,2.9,ml,2.7,"17 km SE of Waynoka, Oklahoma",reviewed,36.4745,-98.7420,6.848,2016-02-01 18:30:44.500,"36.4745,-98.742","[3509323212, 3515123752]",2,2016-02,2016-02-01
4131,us20004wdy,1454363130500,2.5,ml,1.0,"10 km W of Lamont, Oklahoma",reviewed,36.6991,-97.6743,5.599,2016-02-01 21:45:30.500,"36.6991,-97.6743","[3504724516, 3504724444, 3505322795, 350532278...",34,2016-02,2016-02-01
4130,us20004wfa,1454380197600,3.6,ml,2.0,"16 km SE of Waynoka, Oklahoma",reviewed,36.4777,-98.7442,7.137,2016-02-02 02:29:57.600,"36.4777,-98.7442","[3509323212, 3515121599, 3515123752]",3,2016-02,2016-02-02
4129,us20004whq,1454390466800,2.5,ml,NaN,"8 km S of Mulhall, Oklahoma",reviewed,35.9880,-97.4185,5.048,2016-02-02 05:21:06.800,"35.988,-97.4185","[3508323772, 3508323943, 3508323957, 350832277...",35,2016-02,2016-02-02
4128,us20004wih,1454401234600,2.8,ml,1.0,"16 km SE of Waynoka, Oklahoma",reviewed,36.4789,-98.7437,9.937,2016-02-02 08:20:34.600,"36.4789,-98.7437","[3509323212, 3515121599, 3515123752]",3,2016-02,2016-02-02
4127,us20004wjs,1454419945500,3.4,ml,1.0,"17 km SE of Waynoka, Oklahoma",reviewed,36.4770,-98.7416,7.460,2016-02-02 13:32:25.500,"36.477,-98.7416","[3509323212, 3515121599, 3515123752]",3,2016-02,2016-02-02
4126,us20004wrm,1454469212450,2.5,mb_lg,NaN,"9 km NW of Arpelar, Oklahoma",reviewed,34.9910,-96.0509,5.000,2016-02-03 03:13:32.450,"34.991,-96.0509","[3506324359, 3506324315, 3512124652, 350632445...",6,2016-02,2016-02-03


In [332]:
eq_df_20km.to_csv("../oklahoma_earthquakes_largefiles/python_exports/eq_20km.csv", index = False)

In [331]:
fig = px.scatter(eq_df_20km, x="num_wells", y="mag")
fig.show()

In [173]:
api = str(3500300026)

api_list = [3500321242, 3500321328, 3500321404]

date = "2019-02-30 10:46:54.510"

total_sum = 0

# add in when made into function
# if len(api_list) == 0

#     return 0

# else:

for api in api_list:
    month_dict = {"1": "JanVol", "2": "FebVol", "3": "MarVol", "4": "AprVol", 
                      "5": "MayVol", "6": "JunVol", "7": "JulVol", "8": "AugVol", 
                      "9": "SepVol", "10": "OctVol", "11": "NovVol", "12": "DecVol"}

    api = str(api)
    
    month = date[5:7]

    if int(month) == 2:

        year1 = str(date[0:4])
        year2 = str(int(date[0:4]) - 1)
        api_to_check1 = api + "-" + year1
        api_to_check2 = api + "-" + year2

        temp_sum = 0
        month0 = month_dict[str(int(month))]
        month1 = month_dict[str(int(month) - 1)]
        month2 = month_dict[str(int(month) + 10)]

        temp_for_sum = monthly_inj_df.loc[monthly_inj_df["API_year"] == api_to_check1, [month0, month1]]
        temp_sum += (temp_for_sum[month0] + temp_for_sum[month1]).to_list()[0]

        temp_for_sum = monthly_inj_df.loc[monthly_inj_df["API_year"] == api_to_check2, [month2]]
        temp_sum += (temp_for_sum[month2]).to_list()[0]

        total_sum += temp_sum

    elif int(month) == 1:

        year1 = str(date[0:4])
        year2 = str(int(date[0:4]) - 1)
        api_to_check1 = api + "-" + year1
        api_to_check2 = api + "-" + year2

        temp_sum = 0
        month0 = month_dict[str(int(month))]
        month1 = month_dict[str(int(month) + 11)]
        month2 = month_dict[str(int(month) + 10)]

        temp_for_sum = monthly_inj_df.loc[monthly_inj_df["API_year"] == api_to_check1, [month0]]
        temp_sum += (temp_for_sum[month0] + temp_for_sum[month1]).to_list()[0]

        temp_for_sum = monthly_inj_df.loc[monthly_inj_df["API_year"] == api_to_check2, [month1, month2]]
        temp_sum += (temp_for_sum[month2]).to_list()[0]

        total_sum += temp_sum

    elif int(month) == 0:

        year1 = str(int(date[0:4]) - 1)
        api_to_check1 = api + "-" + year1
        api_to_check2 = api + "-" + year2

        temp_sum = 0
        month0 = month_dict[str(int(month) + 12)]
        month1 = month_dict[str(int(month) + 11)]
        month2 = month_dict[str(int(month) + 10)]

        temp_for_sum = monthly_inj_df.loc[monthly_inj_df["API_year"] == api_to_check1, [month0, month1, month2]]
        temp_sum += (temp_for_sum[month0] + temp_for_sum[month1]).to_list()[0]

        total_sum += temp_sum

    else:

        year1 = str(int(date[0:4]) - 1)
        api_to_check1 = api + "-" + year1

        month0 = month_dict[str(int(month))]
        month1 = month_dict[str(int(month) - 1)]
        month2 = month_dict[str(int(month) - 2)]

        temp_for_sum = monthly_inj_df.loc[monthly_inj_df["API_year"] == api_to_check, [month0, month1, month2]]
        temp_for_sum
        temp_sum = (temp_for_sum[month0] + temp_for_sum[month1] + temp_for_sum[month2]).to_list()[0]
        total_sum += temp_sum

print(total_sum)


19549.1400146


'SepVol'

In [333]:
def squared(x):
    '''
    This function returns the square of the input value.
    '''
    return(x**2)

In [ ]:
squared